# Data cleaning - Proyecto Osteosarcopenia
Este notebook realiza la limpieza de datos del estudio sobre osteosarcopenia realizado por el **Dr. Arturo Rizo Topete** en pacientes del municipio de Escuinapa, Sinaloa.  
El archivo original entregado está en formato `.xlsx`, sin tratamiento previo, y contiene aproximadamente 75 columnas con datos clínicos, antropométricos y funcionales.

In [16]:
# Librerías
import pandas as pd
import numpy as np

### 1. Carga de datos
Dado que las dos primeras filas del archivo contenían información no estructurada como ecabezados de tabla, opté por omitirlas, de modo que la carga de datos incluyera únicamente atributos relevantes.

In [17]:
# carga de datos 
path = "../data/raw/tabla osteosarcopenia escui.xlsx"
df = pd.read_excel(path, skiprows=2)

### 2. Limpieza de etiquetado de columnas
* Se realizó una limpieza en los nombres de los atributos: se eliminaron espcios blancos al inicio y final, se sustituyeron espacios internos por guines bajos (`_`), se eliminaron puntos finales y se estandarizó el texto a minúsculas.
* Se eliminaron columnas irrelvantes para los análisis posteriores, ya que algunos eran redundantes o contenían únicamente valores nulos.
* Se renombraron algunos atributos para que fueran más representativos del contenido real o para facilitar su uso durante el análisis, debido a la complejidad de sus nombres originales.

In [18]:
# character cleaning
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace(".", "")
    .str.lower()
)

# column drop
df = df.drop(columns=[
    '#',
    'nombre',
    'rango_de_edad',
    'lugar',
    '3_ms',
    'resultado2',
    'sarcopenia1',
    'grosor',
    'sarcopenia2',
    'medicamento_3',
    'medicamento_4'
])


# column rename
df = df.rename(columns={
    'imc1': 'clas_imc',
    'resultado': 'grasa_resultado',
    'masa_muscular_absoluta,_kg': 'masa_muscular_absoluta',
    'imme:_kg/m2': 'imme',
    'resultado1': 'imme_resultado',
    'puntaje_sarc_-f': 'sarc_f_puntaje',
    'resultado_de_sarc-f': 'sarc_f_resultado',
    'tiempo_de_ejercico': 'tiempo_de_ejercicio',
    'probabilida_de_fractura_por_fragilidad': 'probabilidad_de_fractura_por_fragilidad',
    'probabilida_de_fractura_de_cadera': 'probabilidad_de_fractura_de_cadera'
    
})

### 3. Limpieza de valores

* Dado que el documento original contenía información fuera del estudio en filas muy posteriores, se determinó conservar únicamente las filas correspondientes a los pacientes incluidos en el análisis.
* Se realizaron correcciones concretas en las siguientes columnas:
  - `columna`: Se diseñó una función específica para corregir errores de captura, donde un número iniciaba con punto (`.`), lo cual —según confirmación del creador del estudio— representaba un valor negativo.
  - `altura_en_cm` y `altura_mencionada`: se diseñó una función para corregir automáticamente valores erróneos, como:
    - alturas en metros (`1.45` → `145`)
  - `prueba_de_la_silla`: incluía el valor `"Incapaz"` como texto; se reemplazó por `NaN` para conservar el tipo de dato numérico.
  - `clasificacion_de_estado_fisico`: se eliminó la palabra inicial redundante ("rendimiento") y se conservó únicamente la categoría relevante.
  - `tiempo_de_ejercicio`: se corrigieron errores de digitación, como ausencia de espacios.
  - `nivel_de_sarcopenia`, `medicamento_1`, `medicamento_2`: se normalizaron errores ortográficos y tipográficos.
* Se iteró sobre cada columna de tipo texto para eliminar espacios en blanco al inicio y final, y se estandarizó todo el contenido a minúsculas.


In [19]:
# selección de pacientes del estudio
df = df.iloc[:53, :]

def corregir_puntos_erroneos(x):
    """
    Corrige valores:
    Que empiezan con "." y deberían ser "-"
    """
    try:
        x = str(x).strip()

        if x.startswith('.'):
            partes = x[1:].split('.')
            if len(partes) >= 2:
                return -(float(f"{partes[0]}.{partes[1]}"))
            else:
                return -float(x[1:])
        else:
            return float(x)
    
    except:
        return np.nan


def corregir_altura(x):
    """
    Corrige errores de datos ingresados en la altura:
    por ejemplo "1.45" a "145"
    """
    try:
        valor = x

        if valor < 3:
            return valor * 100
        else:
            return valor

    except:
        return np.nan


# corrección específica de columnas individuales
df['columna'] = df['columna'].apply(corregir_puntos_erroneos)
df['prueba_de_la_silla'] = df['prueba_de_la_silla'].replace('Incapaz', np.nan)
df['altura_en_cm'] = df['altura_en_cm'].apply(corregir_altura)
df['altura_mencionada'] = df['altura_mencionada'].apply(corregir_altura)
df['clasificacion_de_estado_fisico'] = df['clasificacion_de_estado_fisico'].apply(lambda x: x.split(' ', 1)[-1] if isinstance(x, str) else x)
df['tiempo_de_ejercicio'] = df['tiempo_de_ejercicio'].replace({
    '61 -120 mins': '61 - 120 mins',
    '121 -180mins': '121 - 180 mins'
})

df['nivel_de_sarcopenia'] = df['nivel_de_sarcopenia'].replace({
    'Sarcopenoa Leve': 'sarcopenia leve'
})

df['medicamento_1'] = df['medicamento_1'].replace({
    'hipnoticos o ancioliticos': 'hipnoticos o ansioliticos'
})

df['medicamento_2'] = df['medicamento_2'].replace({
    'hipnoticos o ancioliticos': 'hipnoticos o ansioliticos'
})

# estandarización y limpieza de espacios
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip().str.lower()

/var/folders/rd/bxvzw2690fn36hkcfrn53yh80000gn/T/ipykernel_11965/1722507264.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['prueba_de_la_silla'] = df['prueba_de_la_silla'].replace('Incapaz', np.nan)


### 4. Creación de categorías ordenadas.
* Se identificaron las columnas categóricas con un orden jerárquico inherente (por ejemplo: clasificación de imc, tiempo de ejercicio, etc).
* Se definió un diccionario con el orden explícito de las categorías para cada una de estas variables, facilitando su interpretación clínica y futura visualización.
* Si iteró sobre dicho diccionario para convertir automáticamente cada columna en una variable de tipo `Categorical` con orden definido (`ordered=True`), lo que permitirá:
  - Graficar correctamente las categorías en el orden esperado.
  - Realizar comparaciones lógicas y análisis estadísticos entre niveles.
  - Preservar el significado clínico de los rangos en fases posteriores al análisis.

In [20]:
# Ordenamiento de variables por categorias
categorias_ordenadas = {
    'clas_imc': ['adecuado', 'sobrepeso', 'obesidad 1', 'obesidad 2', 'obesidad 3'],
    'grasa_resultado': ['normal', 'bueno', 'alta', 'peligrosamente alta'],
    'imme_resultado': ['normal', 'sarcopenia grado 1', 'sarcopenia grado 2'],
    'estratificacion_de_riesgo': ['baja', 'moderada', 'alta', 'muy alta'],
    'clasificacion_de_estado_fisico' : ['bajo', 'intermedio', 'alto'],
    'fuerza': ['ninguna', 'poca', 'mucha'],
    'caminata': ['ninguna', 'poca', 'mucha'],
    'levantarse': ['ninguna', 'poca', 'mucha'],
    'subir_escaleras': ['ninguna', 'poca', 'mucha'],
    'caidas': ['ninguna', '1 - 3 caidas', '4 o mas caidas'],
    'tiempo_de_ejercicio': ['no', '30 - 60 mins', '61 - 120 mins', '121 - 180 mins','mas de 180 mins'],
    'nivel_de_sarcopenia': ['sin sarcopenia', 'sarcopenia leve', 'sarcopenia moderada', 'sarcopenia severa']
}

for col, orden in categorias_ordenadas.items():
    if col in df.columns:
        df[col] = pd.Categorical(df[col], categories=orden, ordered=True)

### 5. Se exporta el nuevo dataset en formato csv.
Se exporta el DataFrame resultante a un archivo `.csv` en la carpeta `data/cleaned`, conservando únicamente los registros y atributos limpios, listos para su análisis exploratorio.

In [21]:
df.to_csv('../data/cleaned/df_limpio.csv', index=False)